# More Background
<hr/>

<div class="alert alert-info">
[Info]: We use the functionallity provided by <a href="https://www.fuzzingbook.org">The Fuzzingbook</a>. For a more detailed description of Grammars, have a look at the chapter <a href="https://www.fuzzingbook.org/html/Grammars.html">Fuzzing with Grammars</a>.
</div>

<div class="alert alert-success alertsuccess">
[Tip]: To execute the Python code in the code cell below, click on the cell to select it and press <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>

In [ ]:
print("Hello, this is a notebook!")

In [ ]:
from typing import List, Dict, Union, Any, Tuple, Optional
from fuzzingbook import Fuzzer
from fuzzingbook.Grammars import Grammar, Expansion

## Input Languages

All possible behaviors of a program can be triggered by its input.  "Input" here can be a wide range of possible sources: We are talking about data that is read from files, from the environment, or over the network, data input by the user, or data acquired from interaction with other resources.  The set of all these inputs determines how the program will behave – including its failures.  When testing, it is thus very helpful to think about possible input sources, how to get them under control, and _how to systematically test them_.

For the sake of simplicity, we will assume for now that the program has only one source of inputs; this is the same assumption we have been using in the previous chapters, too.  The set of valid inputs to a program is called a _language_.  Languages range from the simple to the complex: the CSV language denotes the set of valid comma-separated inputs, whereas the Python language denotes the set of valid Python programs.  We commonly separate data languages and programming languages, although any program can also be treated as input data (say, to a compiler).  The [Wikipedia page on file formats](https://en.wikipedia.org/wiki/List_of_file_formats) lists more than 1,000 different file formats, each of which is its own language.

To formally describe languages, the field of *formal languages* has devised a number of *language specifications* that describe a language.  *Regular expressions* represent the simplest class of these languages to denote sets of strings: The regular expression `[a-z]*`, for instance, denotes a (possibly empty) sequence of lowercase letters.  *Automata theory* connects these languages to automata that accept these inputs; *finite state machines*, for instance, can be used to specify the language of regular expressions.

Regular expressions are great for not-too-complex input formats, and the associated finite state machines have many properties that make them great for reasoning.  To specify more complex inputs, though, they quickly encounter limitations.  At the other end of the language spectrum, we have *universal grammars* that denote the language accepted by *Turing machines*.  A Turing machine can compute anything that can be computed; and with Python being Turing-complete, this means that we can also use a Python program $p$ to specify or even enumerate legal inputs.  But then, computer science theory also tells us that each such testing program has to be written specifically for the program to be tested, which is not the level of automation we want.

## Grammars

The middle ground between regular expressions and Turing machines is covered by *grammars*.  Grammars are among the most popular (and best understood) formalisms to formally specify input languages.  Using a grammar, one can express a wide range of the properties of an input language.  Grammars are particularly great for expressing the *syntactical structure* of an input, and are the formalism of choice to express nested or recursive inputs.  The grammars we use are so-called *context-free grammars*, one of the easiest and most popular grammar formalisms.

### Rules and Expansions

A grammar consists of a *start symbol* and a set of *expansion rules* (or simply *rules*) which indicate how the start symbol (and other symbols) can be expanded.  As an example, consider the following grammar, denoting a sequence of two digits:

```
<start> ::= <digit><digit>
<digit> ::= 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9
```

To read such a grammar, start with the start symbol (`<start>`).  An expansion rule `<A> ::= <B>` means that the symbol on the left side (`<A>`) can be replaced by the string on the right side (`<B>`).  In the above grammar, `<start>` would be replaced by `<digit><digit>`.

In this string again, `<digit>` would be replaced by the string on the right side of the `<digit>` rule.  The special operator `|` denotes *expansion alternatives* (or simply *alternatives*), meaning that any of the digits can be chosen for an expansion.  Each `<digit>` thus would be expanded into one of the given digits, eventually yielding a string between `00` and `99`.  There are no further expansions for `0` to `9`, so we are all set.

The interesting thing about grammars is that they can be *recursive*. That is, expansions can make use of symbols expanded earlier – which would then be expanded again.  As an example, consider a grammar that describes integers:

```
<start>  ::= <integer>
<integer> ::= <digit> | <digit><integer>
<digit>   ::= 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9
```

Here, a `<integer>` is either a single digit, or a digit followed by another integer.  The number `1234` thus would be represented as a single digit `1`, followed by the integer `234`, which in turn is a digit `2`, followed by the integer `34`.

If we wanted to express that an integer can be preceded by a sign (`+` or `-`), we would write the grammar as

```
<start>   ::= <number>
<number>  ::= <integer> | +<integer> | -<integer>
<integer> ::= <digit> | <digit><integer>
<digit>   ::= 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9
```

These rules formally define the language: Anything that can be derived from the start symbol is part of the language; anything that cannot is not.

### Arithmetic Expressions

Let us expand our grammar to cover full *arithmetic expressions* – a poster child example for a grammar.  We see that an expression (`<expr>`) is either a sum, or a difference, or a term; a term is either a product or a division, or a factor; and a factor is either a number or a parenthesized expression.  Almost all rules can have recursion, and thus allow arbitrary complex expressions such as `(1 + 2) * (3.4 / 5.6 - 789)`.

```
<start>   ::= <expr>
<expr>    ::= <term> + <expr> | <term> - <expr> | <term>
<term>    ::= <term> * <factor> | <term> / <factor> | <factor>
<factor>  ::= +<factor> | -<factor> | (<expr>) | <integer> | <integer>.<integer>
<integer> ::= <digit><integer> | <digit>
<digit>   ::= 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9
```

In such a grammar, if we start with `<start>` and then expand one symbol after another, randomly choosing alternatives, we can quickly produce one valid arithmetic expression after another.  Such *grammar fuzzing* is highly effective as it comes to produce complex inputs, and this is what we will implement in this chapter.

## Representing Grammars in Python

Our first step in building a grammar fuzzer is to find an appropriate format for grammars.  To make the writing of grammars as simple as possible, we use a format that is based on strings and lists.  Our grammars in Python take the format of a _mapping_ between symbol names and expansions, where expansions are _lists_ of alternatives.  A one-rule grammar for digits thus takes the form

In [ ]:
DIGIT_GRAMMAR = {
    "<start>":
        ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
}

We can capture the grammar structure in a _`Grammar`_ type, in which each symbol (a string) is mapped to a list of expansions (strings):

In [ ]:
Grammar = Dict[str, List[Expansion]]

With this `Grammar` type, the full grammar for arithmetic expressions looks like this:

In [ ]:
EXPR_GRAMMAR: Grammar = {
    "<start>":
        ["<expr>"],

    "<expr>":
        ["<term> + <expr>", "<term> - <expr>", "<term>"],

    "<term>":
        ["<factor> * <term>", "<factor> / <term>", "<factor>"],

    "<factor>":
        ["+<factor>",
         "-<factor>",
         "(<expr>)",
         "<integer>.<integer>",
         "<integer>"],

    "<integer>":
        ["<digit><integer>", "<digit>"],

    "<digit>":
        ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
}

Now, let us use the grammar to produce new inputs. For this task, we import the already implemented `GrammarFuzzer` from the fuzzingbook.

In [ ]:
from fuzzingbook.GrammarFuzzer import GrammarFuzzer

<div class="alert alert-info">
[Info]: We use the functionallity provided by <a href="https://www.fuzzingbook.org">The Fuzzingbook</a>. For a more detailed description of the GrammarFuzzer, have a look at the chapter <a href="https://www.fuzzingbook.org/html/GrammarFuzzer.html">Efficient Grammar Fuzzing</a>.
</div>

In [ ]:
# Write a GrammarFuzzer that generates inputs with the EXPR GRAMMAR
g = GrammarFuzzer(EXPR_GRAMMAR)

In [ ]:
# Use g.fuzz to generate arithmetic expressions

for i in range(10):
    print(g.fuzz())

In [ ]:
# show the derivation tree with function display_tree(tree)
from fuzzingbook.GrammarFuzzer import display_tree

print(g.fuzz())
print(type(g.derivation_tree))

# show the derivation tree with function display_tree(tree)
display_tree(g.fuzz_tree())

### Parsing Inputs

In [ ]:
# We can also parse new inputs with the Grammar
from fuzzingbook.Parser import EarleyParser

parser = EarleyParser(EXPR_GRAMMAR)
for tree in parser.parse('1 * 2'):
    display(tree)

In [ ]:
# Show derivation tree for 1 * 2
display_tree(tree)

In [ ]:
# Turn derivation tree into arithmetic expression with tree_to_string(tree)
from fuzzingbook.GrammarFuzzer import tree_to_string

tree_to_string(tree)

In [ ]:
assert tree_to_string(tree) == '1 * 2'

## "Uncommon Input Generation"

“Uncommon inputs”: Learning from common samples,
the resulting inverted grammar describes in turn the
distribution of legal, but uncommon inputs. This is useful
for completing test suites by testing uncommon features.

In [ ]:
from fuzzingbook.ProbabilisticGrammarFuzzer import ProbabilisticGrammarMiner, ProbabilisticGrammarFuzzer
from fuzzingbook.Parser import EarleyParser

probabilistic_grammar_miner = ProbabilisticGrammarMiner(
    EarleyParser(EXPR_GRAMMAR))

learned_probabilistic_grammar = probabilistic_grammar_miner.mine_probabilistic_grammar(["1 + (2 * 3)", "4 * 9"])

fuzzer = ProbabilisticGrammarFuzzer(learned_probabilistic_grammar)

for i in range(10):
    print(fuzzer.fuzz())

<div class="alert alert-danger">
[Info]: In their paper "<a href="https://publications.cispa.saarland/3167/7/inputs-from-hell.pdf">Inputs from Hell</a>", Sorumekun et al. use a different 'inverting' strategy.</a>
</div>

In [ ]:
from fuzzingbook.ProbabilisticGrammarFuzzer import invert_probs

import copy

def exp_prob(expansion: Expansion) -> float:
    """Return the options of an expansion"""
    return exp_opt(expansion, 'prob')

def invert_expansion(expansion: List[Expansion]) -> List[Expansion]:
    def sort_by_prob(x: Tuple[int, float]) -> float:
        index, prob = x
        return prob if prob is not None else 0.0

    inverted_expansion: List[Expansion] = copy.deepcopy(expansion)
    indexes_and_probs = [(index, exp_prob(alternative))
                         for index, alternative in enumerate(expansion)]
    print(indexes_and_probs)
    indexes_and_probs.sort(key=sort_by_prob)
    indexes = [i for (i, _) in indexes_and_probs]

    for j in range(len(indexes)):
        k = len(indexes) - 1 - j
        # print(indexes[j], "gets", indexes[k])
        inverted_expansion[indexes[j]][1]['prob'] = expansion[indexes[k]][1]['prob']

    return inverted_expansion

In [ ]:
inverted_grammar = invert_probs(learned_probabilistic_grammar)
display(inverted_grammar)

In [ ]:
inverted_fuzzer = ProbabilisticGrammarFuzzer(inverted_grammar)

In [ ]:
for i in range(10):
    print(inverted_fuzzer.fuzz())